In [62]:
import selenium
from selenium import webdriver
from selenium.common import exceptions

In [20]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import time
import re

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# make to download the correct chrome driver, update path to its directory
wd = webdriver.Chrome(executable_path='./chromedriver.exe')
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=San%20Francisco%20Bay%20Area&locationId=&geoId=90000084&sortBy=R&f_TPR=&f_E=2%2C3&position=1&pageNum=0'
wd.get(url)
wd.maximize_window()

In [4]:
# convert str result of job counts into int
no_of_jobs = wd.find_element_by_css_selector('h1>span').get_attribute('innerText')
no_of_jobs_str = no_of_jobs
no_of_jobs_str = no_of_jobs_str[:-1]
no_of_jobs_int = int(no_of_jobs_str.replace(',',''))
no_of_jobs_int

3000

In [37]:
# expanding the listing to get a total of no_of_jobs_in posts
i = 2

while i <= int(no_of_jobs_int/25) + 1:
# while i <= int(1000/25) + 1: # just want about 1000 postings
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    i = i+1
    try:
        wd.find_element_by_class_name('infinite-scroller__show-more-button.infinite-scroller__show-more-button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

In [46]:
def jobIsNotWanted(title):
    if re.search('analyst', title.lower()):
        return False
    return True

In [157]:
# expanding the jobs listing

def expandList():
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')

    try:
        wd.find_element_by_class_name('infinite-scroller__show-more-button.infinite-scroller__show-more-button').click()
        time.sleep(5)
    except:
        pass

In [159]:
# expand job details and get text
def getJobDesc():
    job_desc = ''
    detail_panel = wd.find_element_by_class_name('two-pane-serp-page__detail-view')
    core_section = detail_panel.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div[1]/section[1]')
    
    # handling different details panels
    try:
        show_more_button = core_section.find_element_by_tag_name('div>div>section>button')
        show_more_button.click()
        job_desc = core_section.find_element_by_tag_name('div>section>div').text
    except exceptions.NoSuchElementException:                                                  
        show_more_button = core_section.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/button[1]')
        show_more_button.click()                              
        job_desc = core_section.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[2]/div').text
    except exceptions.ElementNotInteractableException as err:
        print(err)
        pass
    except:
        pass
    
    return job_desc

In [152]:
# load up existing df is there is one
job_id = []
job_title = []
company_name = []
city = []
state = []
location = []
description = {}
# job_df = pd.read_csv('jobs.csv')
# print(job_df.columns)

# job_id = job_df['id'].values.tolist()
# job_title = job_df['title'].values.tolist()
# company_name = job_df['company_name'].values.tolist()
# city = job_df['city'].values.tolist()
# state = job_df['state'].values.tolist()

In [32]:
# use this if we only expand as we go

# looping over the job listing until we have 100 unique job in list
# or exceeding the amount of available posting
jobs = []
curr_job_count = 0

while len(job_id) < 100:
    
    curr_job_count = len(jobs)
    
    # get a list of jobs
    job_listing = wd.find_element_by_class_name('jobs-search__results-list')
    jobs_posts = job_listing.find_elements_by_tag_name('li')
    # only indexing jobs we have not went over yet
    jobs = jobs_posts[curr_job_count:len(jobs_posts)]
    
    for job in jobs:
        # click on each li ele to populate the job card 
        job.click()
        time.sleep(0.5)
        
        url = wd.current_url
        url_parts = url.split('JobId=')
        if(url_parts[1] in job_id): # move on to next iteration if already exist
            continue
        job_id.append(url_parts[1])
            
        # get job title
        title = job.find_element_by_css_selector('div>h3')
        job_title.append(title.text)

        # get the company name
        company = job.find_element_by_css_selector('div>h4')
        company_name.append(company.text)

        # job location
        job_location = job.find_element_by_class_name('job-search-card__location')
        location.append(job_location.text)
        
        

    expandList()

In [156]:
##### use this block if list expanded to total posting before hand

job_listing = wd.find_element_by_class_name('jobs-search__results-list')
jobs = job_listing.find_elements_by_tag_name('li')

for job in jobs:
    # click on each li ele to populate the job card 
    job.click()
    time.sleep(.5)
    # get job id and title, filtering out existing and non-desirable job based on title
    url = wd.current_url
    url_parts = url.split('JobId=')
    title = job.find_element_by_css_selector('div>h3')
    # move on to next iteration if already exist and job is not wanted
    if(url_parts[1] in job_id or jobIsNotWanted(title.text)): 
        continue
        
    job_id.append(url_parts[1])
    job_title.append(title.text)

    # get the company name
    company = job.find_element_by_css_selector('div>h4')
    company_name.append(company.text)

    # job location
    job_location = job.find_element_by_class_name('job-search-card__location')
    location.append(job_location.text)
    
    # job description
    desc = getJobDesc()
    description[url_parts[1]] = desc

    time.sleep(1)

In [9]:
# scrubbing null values for job level
# missing_seniority_level = seniority_level
# missing_seniority_level = [i if len(i) > 0 else 'Not Applicable' for i in seniority_level]

In [10]:
# separate city and state
# state = []
# city = []

# for i in range(len(location)):
#     state.append(location[i][-2:])
#     city.append(location[i][:-4])

In [11]:
# replacing , in company name, just personal ref
# cleaned_company_name = [name.replace(',','-') for name in company_name]

In [166]:
job_df = pd.DataFrame()

job_dict = {
    'id': job_id,
    'title': job_title,
    'company_name': company_name,
    'location': location
}

job_df = pd.DataFrame(job_dict)
job_df.to_csv('jobs.csv', index = False)

In [26]:
# TODO: 
# description, combine all substirngs into a giant string
# sql, python, R, Tableau, BI, excel, looker, powerpoint, SAS, ETL, Jupyter Notebooks

In [189]:
job_desc_df = pd.DataFrame.from_dict({'id': description.keys(), 'description':description.values()})
job_desc_df.to_csv('jobs_desc.csv', index = False)